In [1]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/mihkelmariuszjezierski/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
from datasets import load_dataset

# Load dataset from JSON file
dataset = load_dataset('json', data_files='../data/processed/DEV/dataset.json')

# Split the dataset into train and eval (90% train, 10% eval)
train_dataset = dataset['train'].train_test_split(test_size=0.1)["train"]
eval_dataset = dataset['train'].train_test_split(test_size=0.1)["test"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")

Generating train split: 0 examples [00:00, ? examples/s]

Train dataset size: 1380
Eval dataset size: 154


In [7]:
model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-3b")
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-3b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [59]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,  # Lowered batch size
    per_device_eval_batch_size=1,   # Lowered eval batch size
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=3,
)

/Users/mihkelmariuszjezierski/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [60]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [61]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

In [62]:
def formatting_prompts_func(datapoint):
    question = datapoint['question']
    query = datapoint['SQL']
    database_schema = datapoint['database_schema']
    prompt = f"""Given the following SQL tables, your job is to generate the Sqlite SQL query given the user's question.
Put your answer inside the ⁠```sql and ```⁠ tags.
{database_schema}
###
Question: {question}

⁠```sql
{query} ;
```
<|EOT|>
"""
    
    return tokenize(prompt)

In [63]:
tokenized_train_dataset = train_dataset.map(formatting_prompts_func)
tokenized_val_dataset = eval_dataset.map(formatting_prompts_func)

Map:   0%|          | 0/1380 [00:00<?, ? examples/s]

In [69]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Training dataset
    eval_dataset=tokenized_val_dataset,    # Evaluation dataset (if applicable)
)

RuntimeError: MPS backend out of memory (MPS allocated: 9.05 GB, other allocations: 384.00 KB, max allowed: 9.07 GB). Tried to allocate 67.50 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
# Start training
trainer.train()
trainer.save_model("/Users/mikaumana/Downloads")